In [289]:
import pandas as pd 
import numpy as np

#Reading Cases

FBL13= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\FBL-38.xlsm',header=1,sheet_name='BRD')
HBL134= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\HBL-134.xlsm',header=1,sheet_name='BRD')
MBL61= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\MBL-61.xlsm',header=1,sheet_name='BRD')
HMB69 = pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\HMB-69.xlsm',header=1,sheet_name='BRD')
#Combining cases
all_cases = pd.concat((FBL13,HBL134,MBL61,HMB69),axis=0)
#Reading Rates
rates = pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\Rates.xlsx') 

In [290]:
#Reading BCA
BCAFBL13= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\FBL-38.xlsm',header=0,sheet_name='BCA')
BCAHBL134= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\HBL-134.xlsm',header=0,sheet_name='BCA')
BCAMBL61= pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\MBL-61.xlsm',header=0,sheet_name='BCA')
BCAHMB69 = pd.read_excel(r'C:\Users\husnain110551\Desktop\Review\RC\HMB-69.xlsm',header=0,sheet_name='BCA')
#Combining BCA
all_bca = pd.concat((BCAFBL13,BCAHBL134,BCAMBL61,BCAHMB69),axis=0,sort=True)

In [291]:
#melting df(Rates)
melted_rates = (rates.melt('Realization Date', var_name='Currency', value_name='SBP DMMD WAR'))

In [292]:
#Return Rates
rates_data = pd.merge(all_cases,melted_rates,on=('Realization Date','Currency'),how='left')

In [293]:
#Removing duplicates from BCA
all_bca.drop_duplicates(keep='first',inplace=True)

In [294]:
#renaming column (DOR)
all_bca.rename(columns={'Date Of Realization':'Realization Date'}, inplace=True)

In [295]:
#renaming column (EFE)
all_bca.rename(columns={'EFE Number':'EFE / Manual E-Form / Gate Pass No.'}, inplace=True)

In [296]:
#Merging  all_cases with selected columns of bca to cross check DOR based on EFE
cross = all_cases.merge(all_bca[['EFE / Manual E-Form / Gate Pass No.','Realization Date',"BCA(FC)",'Foreign Bank Charges','Agent/Brokerage Commision']],how='left',on=('EFE / Manual E-Form / Gate Pass No.','Realization Date'),indicator="BCA Availability")

In [297]:
#Dropping duplicates from CASES and BCA merger.
cross.drop_duplicates(subset='Sr.',keep='first',inplace=True)

In [298]:
#merging all cases and rates based on identical columns
all_cases = cross.merge(melted_rates, how='left', indicator='Rate Check')
#applying mask
mask = all_cases['Rate Check'].eq('both')

In [299]:
#Replacing mask
all_cases['Rate Check'] = np.where(mask, 'Rate Match','FX Rate Mismatch')

In [300]:
#Checking Realized Value
all_cases['RV Check'] = all_cases['Value Realized'] >((all_cases['BCA(FC)']+all_cases['Foreign Bank Charges']+all_cases['Agent/Brokerage Commision'])*0.90)

In [301]:
all_cases.to_excel('switch.xlsx',index=0)